__Imports__

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
import tensorflow_datasets as tfds

### Load in the data__
* Splits the data
* Shuffle the data

__Batch_size=-1 to get the full dataset in NumPy arrays from the returned tf.Tensor object__

In [ ]:
mnist_test,mnist_val,mnist_train = tfds.load(name="kmnist",
                                             split=["train[:10%]", "train[10%:25%]", "train[25%:]"],
                                             batch_size=-1,
                                             shuffle_files=True
                                            ) 

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

__tfds.as_numpy return a generator that yields NumPy array records out of a tf.data.Dataset__

In [ ]:
# tfds.as_numpy return a generator that yields NumPy array records out of a tf.data.Dataset
mnist_train = tfds.as_numpy(mnist_train) 
mnist_val = tfds.as_numpy(mnist_val) 
mnist_test = tfds.as_numpy(mnist_test)

__Seperate the x and y__

In [ ]:
x_train, y_train = mnist_train["image"], mnist_train["label"]
x_valid,y_valid = mnist_val["image"], mnist_val["label"]
x_test, y_test = mnist_test["image"], mnist_test["label"]

__Check lengths__

In [ ]:
print("Train set size: ", len(x_train))
print("Valid set size: ", len(x_valid))
print("Test set size: ", len(x_test))

__Data Normalization__

In [ ]:
# Normalize values
x_train = x_train/255
x_valid = x_valid/255
x_test = x_test/255

__Visualize the Data__

In [ ]:
from matplotlib import pyplot as plt
fig = plt.figure(figsize=(6, 6))  # figure size in inches
fig.subplots_adjust(left=0, right=1, bottom=0, top=1, hspace=0.05, wspace=0.05)

for i in range(64):
    ax = fig.add_subplot(8, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(x_train[i], cmap=plt.cm.binary, interpolation='nearest')
    # label the image with the target value
    ax.text(0, 7, str(y_train[i]))

### Processing Data

__Accuracy Function__

In [ ]:
def rounded_accuracy(y_true, y_pred):
    return keras.metrics.binary_accuracy(tf.round(y_true), tf.round(y_pred))

__Always a good habit to randomized seeds__

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)

__Autoencoder Model__

__Encoder__

In [ ]:
stacked_encoder = keras.models.Sequential([
    keras.layers.Input(shape=(28,28,1)),
    keras.layers.Conv2D( filters=16, kernel_size=3, 
                        padding="SAME",
                        activation="selu" ),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D( filters=32, kernel_size=3, 
                        padding="SAME",
                        activation="selu" ),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Conv2D( filters=64, kernel_size=3, 
                        padding="SAME",
                        activation="selu" ),
    keras.layers.MaxPool2D(pool_size=2)
])
stacked_encoder.summary()

__Decoder__

In [ ]:
stacked_decoder = keras.models.Sequential([
    # padding="VALID" since 3*2 !=7
    keras.layers.Conv2DTranspose( 32, kernel_size=3, 
                                 strides=2, padding="VALID",
                                 activation="selu", input_shape=[3,3,64]), #==> (None, 7, 7, 32)
    # 7x2 (strides)=14 and padding=0
    keras.layers.Conv2DTranspose( 16, kernel_size=3, strides=2, padding="SAME",
                                 activation="selu" ), #==> (None, 14, 14, 16)
    # 14x2(strides)=28
    keras.layers.Conv2DTranspose( 1, kernel_size=3, strides=2, padding="SAME", 
                                 activation="sigmoid" ), #==> (None, 28, 28, 1)
    keras.layers.Reshape([28,28])
])
stacked_decoder.summary()

__Now connect them together__

In [ ]:
stacked_ae = keras.models.Sequential([stacked_encoder, stacked_decoder])

__Binary cross-entropy loss: each pixel(or each label) intensity represents the probability that the pixel should so model converges faster__

In [ ]:
stacked_ae.compile(loss="binary_crossentropy",
                   optimizer=tf.keras.optimizers.SGD(learning_rate=1.5), 
                   metrics=[rounded_accuracy])

__Using x_train as both the inputs and the targets and similarly, we use x_valid as both the validation inputs and targets__


In [ ]:
history = stacked_ae.fit(x_train, x_train,
                         epochs=20,
                         validation_data=(x_valid, x_valid))

### Reconstruction Zone & Analysis

__Function to show plots__

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="binary")
    plt.axis("off")

__Function to Reconstruction Images given a model__

In [ ]:
def show_reconstructions(model, images=x_valid, n_images=5):
    reconstructions = model.predict(images[:n_images])
    fig = plt.figure(figsize=(n_images * 1.5, 3))
    for image_index in range(n_images):
        plt.subplot(2, n_images, 1 + image_index)
        plot_image(images[image_index])
        plt.subplot(2, n_images, 1 + n_images + image_index)
        plot_image(reconstructions[image_index])


__Top is Original & Bottom is Reconstructed__

In [ ]:
show_reconstructions(stacked_ae)

__Analysis__

__t-SNE is based on probability distributions with random walk on neighborhood graphs to find the structure
within the data.__

In [ ]:
np.random.seed(42)
from sklearn.manifold import TSNE
X_valid_compressed = stacked_ae.predict(x_valid)
nsamples, nx, ny,oth = X_valid_compressed.shape
X_valid_compressed = X_valid_compressed.reshape((nsamples,nx*ny))
tsne = TSNE()
X_valid_2D = tsne.fit_transform(X_valid_compressed)
X_valid_2D = (X_valid_2D - X_valid_2D.min()) / (X_valid_2D.max() - X_valid_2D.min())

__t-SNE Plot__

In [ ]:
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], c=y_valid, s=10, cmap="tab10")
plt.axis("off")
plt.show()

__t-SNE with Images__

In [ ]:
import matplotlib as mpl
plt.figure(figsize=(10, 8))
cmap = plt.cm.tab10
plt.scatter(X_valid_2D[:, 0], X_valid_2D[:, 1], c=y_valid, s=10, cmap=cmap)
image_positions = np.array([[1., 1.]])
for index, position in enumerate(X_valid_2D):
    dist = np.sum((position - image_positions) ** 2, axis=1)
    if np.min(dist) > 0.02: # if far enough from other images
            image_positions = np.r_[image_positions, [position]]
            imagebox = mpl.offsetbox.AnnotationBbox(
                mpl.offsetbox.OffsetImage(x_valid[index], cmap="binary"),
                position, bboxprops={"edgecolor": cmap(y_valid[index]), "lw": 2})
    plt.gca().add_artist(imagebox)
plt.axis("off")
plt.show()
